---
# Running Molecular Dynamics simulations in Python
---

## 1. Simulation of dynamics:

The II Newton's law  connects the kinematics (movements) of the body with its mechanics (total force acting on it) and defines its dynamic evoltion: 

$$m\frac{d^2r(t)}{dt^2} = F = - \nabla{U(r)},$$

where $m$ is the mass, $r$ is the coordinate, $F$ is the force and $U(r)$ is the potential energy depending only on the position of the body. 
If one knows the forces acting upon the body, one can find the position of the body at any moment $r(t)$, i.e. predict its dynamics. It can be done by solving Newton's equation of motion. It is a second order ODE that can be numerically integrated with a simple Euler algorithm:

$$ v_{n+1} = v_n + \frac{F_n}{m} dt$$
$$ r_{n+1} = r_n + v_{n+1} dt$$

Here we discretize time with time step $dt$, so $t_{n+1} = t_n + dt$, and $r_{n} = r(t_n)$, $v_{n} = v(t_n)$, where $n$ is the timestep number.

### Example 1.1. Simulation of a projectile on Earth.
We want to know the dynamics of a green apple ($m = 0.3$ kg) tossed horizontally at 10 cm/s speed from the top of the CN tower (553 m) for the first 10 seconds.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(6,6))
ax.set(xlim=(-2, 2), ylim=(0, 600), ylabel='Height, meters', title='Apple falling from CN tower')

# parameters of the problem
T = 10. #s
m = 0.3 #kg
g = 9.8 #m/s^2
v0x = -0.1 #m/s
H = 553. #m

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem
v = np.zeros((N+1, 2))
r = np.zeros((N+1, 2))
f = np.zeros((N+1, 2))

# initial conditions:
r[0] = np.array([0., H])
v[0] = np.array([-v0x, 0.])

# the only force is gravity
f[:] = np.array([0., -m * g])

## Run dynamics:
for n in range(N):
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing and animating 
scat = ax.scatter(r[0,0], r[0,1], marker='o', c='g', s=200)

def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, animate, frames=N)
## this function will create a lot of *.png files in a folder 'CNtower_frames'
ani.save('CNtower.html', writer=animation.HTMLWriter(fps= 1//dt))
plt.close()
#ani.save('CNtower.mp4', fps= 1//dt)

Let's visualize the dynamics using embedded HTML. It's interactive and you play a movie step by step:

In [2]:
from IPython.display import HTML
HTML('CNtower.html')

When a closed system of particles are interacting through pairwise potentials, the force on each pacticle $i$ depends on its position in respect to other pacrticles $j$:

$$m_i\frac{d^2r_i(t)}{dt^2} = \sum_jF_{ij}(t) = -\sum_j\nabla_i{U(|r_{ij}(t)|)}$$

where $r_{ij}$ is the distance between particle $i$ and $j$, and $i,j \in (1,N)$.

### Example 1.2. Simulation of 3-body problem with Hooke's law:

We want to know the dynamics of 3 particles $m = 1$ kg connected to each other with invisible springs with $K_s = 5$ N/m, and $r_0 = 1$  meter initially located at (0, 2), (2, 0) and (-1, 0) on the 2D plane for the first 10 seconds.

Pairwise potential is (Hooke's Law)$$U(r_{ij}) = \frac{K_s}{2}(r_{ij} - r_0)^2$$
Minus gradient is a force from $j$-th upon $i$-th: $$\mathbf{F_{ij}} = - \nabla_i{U(r_{ij})} = - K_s (r_{ij} - r_0) \nabla_i r_{ij} = - K_s (r_{ij} - r_0) \mathbf{r_{ij}} / r_{ij}$$


In [3]:
# Setup the figure and axes...
fig, ax = plt.subplots(figsize=(6,6))
ax.set(xlim=(-3.5, 3.5), ylim=(-3.5, 3.5), ylabel='meters', xlabel='meters', title='3-Body problem')

# parameters of the problem
T = 10. #s
m = 1.0 #kg
ks = 5 #N/m
r0 = 1.

# setting a timestep to be 50 ms
dt = 0.05 #s
N = int(T / dt)

# Allocating arrays for 2D problem
v = np.zeros((N+1, 3, 2))
r = np.zeros((N+1, 3, 2))
f = np.zeros((N+1, 3, 2))

# initial conditions:
r[0,0] = np.array([0., 2.])
r[0,1] = np.array([2., 0.])
r[0,2] = np.array([-1., 0.])

def compute_forces(n):
    '''The function computes forces on each pearticle at time step n'''
    for i in range(3):
        for j in range(3):
            if i != j:
                rij = r[n,i] - r[n,j]
                rij_abs = np.linalg.norm(rij)
                f[n, i] -= ks * (rij_abs - r0) * rij / rij_abs 
## Run dynamics:
for n in range(N):
    compute_forces(n)
    v[n+1] = v[n] + f[n]/m * dt
    r[n+1] = r[n] + v[n+1] * dt

## drawing and animating 
scat = ax.scatter(r[0,:,0], r[0,:,1], marker='o', c=['b', 'k', 'r'], s=1000)

def animate(i):
    scat.set_offsets(r[i])

ani = animation.FuncAnimation(fig, animate, frames=N)
plt.close()
## this function will create a lot of *.png files in a folder '3Body_frames'
ani.save('3body.html', writer=animation.HTMLWriter(fps= 1//dt))

Again, looking at the trajectory representation in real time:

In [4]:
HTML('3body.html')

## 2. Proteins:

## 3. Molecular dynamics of proteins:
[**Molecular dynamics (MD)**](https://en.wikipedia.org/wiki/Molecular_dynamics) is a computer simulation method for studying the physical movements of atoms and molecules. The atoms and molecules are allowed to interact for a fixed period of time, giving a view of the dynamic evolution of the system. In the most common version, the trajectories of atoms and molecules are determined by numerically solving Newton's equations of motion for a system of interacting particles, where forces between the particles and their potential energies are often calculated using interatomic potentials or molecular mechanics force fields. 